In [1]:
import re
import os
import sys
import csv
import copy
import random
import numpy as np 
import pandas as pd 
from Bio.Seq import Seq
from collections import Counter, OrderedDict, namedtuple
from prettytable import PrettyTable

import importlib
# importlib.reload(sys.modules['Evaluation.RepeatEvaluation'])
from Util.FragmentUtil import fragmentLenPlot
from Evaluation.RepeatEvaluation import getConsensusSeqIds, getDfamConSeqData, getOutputMatchPosition, outputPositionBucketClassifier, checkDfamMatch
from Util.SeqUtil import seqInfo, parseFasta, parseSeq, parseSeqByCutter
from DataInfo import currDataset, datasetPath, matchPattern, cutter, cutterLen, fragmentN, commonCount
from RepeatFinder import findRepeatSeqs, integrateRepeatInfo, getIRComb, evaluateRepeat, generateIROutputFile, checkTandemRepeatExist, generateTROutputFile, generateFragmentOutputFile

In [2]:
parseFastaSeqs = parseFasta(currDataset, datasetPath, matchPattern, matchMode = False)
seqInfo(currDataset, parseFastaSeqs)

...start parsing chr1.fa fasta file ...
...cost8.366072177886963 sec to parse fasta file ...
chr1.fa dataset
 number of sequence:1
 total length:248956422



In [3]:
fragmentsLenList, fragmentsSeqList = parseSeqByCutter(parseFastaSeqs)

...start parse seq by cutter: GATC
...cost 2.6790308952331543 sec to cut sequence


In [4]:
repeatFragNLenList, repeatFragNPositionDict = findRepeatSeqs(fragmentsLenList)

... start finding repeat seq ...
...cost1.2262678146362305 sec to finding repeat seq  ...


In [5]:
# tandem Repeat in N fragment
tandemRepeatLenList = list(filter(checkTandemRepeatExist, repeatFragNLenList))
tandemRepeatInfoList = integrateRepeatInfo(fragmentsSeqList, fragmentsLenList, tandemRepeatLenList, repeatFragNPositionDict, repeatType=1)
generateTROutputFile(tandemRepeatInfoList, matchRatioOfSum=0.4)

In [5]:
# interspersed repetitive sequences
repeatInfoList = integrateRepeatInfo(fragmentsSeqList, fragmentsLenList, repeatFragNLenList, repeatFragNPositionDict, repeatType=2)
seqPermutation = getIRComb(repeatInfoList)
generateIROutputFile(seqPermutation, matchRatioOfSum=0.4)

In [8]:
# From Evaluation
OutputMatchPositionList = getOutputMatchPosition(repeatInfoList)
outputPositionLookupDic = outputPositionBucketClassifier(OutputMatchPositionList)

In [10]:
dfamDatasetMatchList = checkDfamMatch(outputPositionLookupDic, OutputMatchPositionList)